In [ ]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, RobertaTokenizer, BertModel, AdamW, RobertaModel, BertTokenizerFast, RobertaTokenizerFast, get_linear_schedule_with_warmup
from tqdm import tqdm
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
import torch
import pdb
from tqdm import tqdm
import torch.nn as nn
import pickle
import argparse
import os
import time
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_class_weight as cls_weigh
from warnings import simplefilter
from egtea_finetuning import compute_performance
import gc
gc.disable()

### Many-shot classes

In [ ]:
labels = pd.read_csv('./../data/epic55_action_seq/actions.csv')
many_shot_labels = pd.read_csv('./../data/epic55_action_seq/EPIC_many_shot_action.csv')
many_shot_labels = many_shot_labels.merge(labels, left_on=['verb_class', 'noun_class'], right_on=['verb', 'noun']).id.to_numpy()
labels_mapping = {x['id']: {'label': x['action']} for idx, x in labels.iterrows() }
labels_mapping_manyshot = {x['id']: {'label': x['action']} for idx, x in labels.iterrows() if x['id'] in many_shot_labels}

file_pi = open(os.path.join('./../data/epic55_action_seq', 'Manyshot_actions.pkl'), 'wb') 
pickle.dump(many_shot_labels, file_pi)
file_pi.close()

with open(os.path.join('./../data/epic55_action_seq', 'Manyshot_actions.pkl'), "rb") as input_file:
    many_shot_labels = pickle.load(input_file)

### <u>Performance computation for both teacher & student</u>

In [ ]:
def compute_epic_metrics(path, teacher = True):
    with open(path, "rb") as input_file:
        epic_pred = pickle.load(input_file)

    if teacher == False:
        logit = epic_pred['logits']
        pred = logit.argmax( axis  = 1)
        target = epic_pred['target']
    else:
        target = np.array([dict['target'] for dict in epic_pred])
        logit = np.concatenate([dict['logit_action'].reshape(1, -1) for dict in epic_pred])
        pred = np.argmax(logit, axis = 1)

    K = 5
    class_recall_list= []
    class_recall_dict = {}

    acc = np.mean(pred == target)
    top_K_acc = np.mean([t in (-p).argsort()[:K] for t, p in zip(target, logit)])

    many_shot_labels_in_test = np.array(list(set(many_shot_labels).intersection(set(target))))
    for c in many_shot_labels_in_test:
        target_subset = target[target == c]
        logit_subset = logit[target == c]
        no_of_class_instances = len(target_subset)

        recall = np.mean([t in (-l).argsort()[:K]  for t, l in zip(target_subset, logit_subset)])
        class_recall_dict[c] = recall

        class_recall_list.append(recall)

    many_shot_recall = np.mean(class_recall_list)
    res_dict = {'Top_1_acc': acc, 'top_5_acc' :top_K_acc, 'm_shot_cm_recall' : many_shot_recall}
    print(path)

    conf_matrix = confusion_matrix(target, pred)
    class_wise_acc = conf_matrix.diagonal()/conf_matrix.sum(1)
    mean_class_acc = np.nanmean(class_wise_acc)

    print(f'Top 1 accuracy is :{mean_class_acc}')
    print(f'Top 5 accuracy is :{acc}, top-5 accuracy is :{top_K_acc}, many-shot top@5 class-mean-recall : {many_shot_recall}\n')
    return res_dict, class_recall_dict

### Generating metrics for teachers that were reported in the paper

In [11]:
teacher_pred_path = "epic"   # epic/egtea
model_name = ['alberta', 'bert', 'roberta', 'electra', 'distillbert']
for model in model_name:
    file_name = f'./../teacher_student_Predictions/teacher/{teacher_pred_path}/{model}_E_7_B_16_T_False_W_False_chpk_1MRecipe_MT_True_hL_5.pkl'
    # file_name = f'./../teacher_student_Predictions/teacher/{teacher_pred_path}/{model}_E_5_T_False_W_True_chpk_1MRecipe_MT_True_hL_15_SS_True_GS_True.pkl'
    compute_epic_metrics(path=file_name, teacher=True)

./../teacher_student_Predictions/teacher/epic/alberta_E_7_B_16_T_False_W_False_chpk_1MRecipe_MT_True_hL_5.pkl
Top 1 accuracy is :0.05919782889020815
Top 5 accuracy is :0.1773448483631251, top-5 accuracy is :0.35930909821249246, many-shot top@5 class-recall : 0.217873340836684



/home/sayontan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


./../teacher_student_Predictions/teacher/epic/bert_E_7_B_16_T_False_W_False_chpk_1MRecipe_MT_True_hL_5.pkl
Top 1 accuracy is :0.05811610043532304
Top 5 accuracy is :0.185378590078329, top-5 accuracy is :0.36171922072705365, many-shot top@5 class-recall : 0.21433124024736552



/home/sayontan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


./../teacher_student_Predictions/teacher/epic/roberta_E_7_B_16_T_False_W_False_chpk_1MRecipe_MT_True_hL_5.pkl
Top 1 accuracy is :0.0604298595706115
Top 5 accuracy is :0.1847760594496887, top-5 accuracy is :0.3647318738702551, many-shot top@5 class-recall : 0.22412845956126234



/home/sayontan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


./../teacher_student_Predictions/teacher/epic/electra_E_7_B_16_T_False_W_False_chpk_1MRecipe_MT_True_hL_5.pkl
Top 1 accuracy is :0.0395415821617073
Top 5 accuracy is :0.17091785499096204, top-5 accuracy is :0.3225547298654348, many-shot top@5 class-recall : 0.15183260971411158



/home/sayontan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


./../teacher_student_Predictions/teacher/epic/distillbert_E_7_B_16_T_False_W_False_chpk_1MRecipe_MT_True_hL_5.pkl
Top 1 accuracy is :0.06037388959043782
Top 5 accuracy is :0.19100220927897169, top-5 accuracy is :0.36573609158465553, many-shot top@5 class-recall : 0.21555259038307426



/home/sayontan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in true_divide


### Generating metrics for students that were reported in the paper

In [12]:
student_pred_path = "egtea"   # epic/egtea
root = f'./../teacher_student_Predictions/student/{student_pred_path}'

for model in os.listdir(root) :
    print(model)
    file_name = os.path.join(root, model)
    compute_epic_metrics(path=file_name, teacher=False)

pretrained_alberta_kldiv_wt150_t10_preds.pickle
./../teacher_student_Predictions/student/egtea/pretrained_alberta_kldiv_wt150_t10_preds.pickle
Top 1 accuracy is :0.36099907860155306
Top 5 accuracy is :0.4520276953511375, top-5 accuracy is :0.7472799208704253, many-shot top@5 class-recall : 0.758117618356589

pretrained_bert_kldiv_wt150_t10_preds.pickle
./../teacher_student_Predictions/student/egtea/pretrained_bert_kldiv_wt150_t10_preds.pickle
Top 1 accuracy is :0.3556648169387721
Top 5 accuracy is :0.44807121661721067, top-5 accuracy is :0.7472799208704253, many-shot top@5 class-recall : 0.7740575702891879

baseline_preds.pickle
./../teacher_student_Predictions/student/egtea/baseline_preds.pickle
Top 1 accuracy is :0.34866889151850244
Top 5 accuracy is :0.4352126607319486, top-5 accuracy is :0.7453016815034619, many-shot top@5 class-recall : 0.7504370663378016

pretrained_electra_kldiv_wt150_t10_preds.pickle
./../teacher_student_Predictions/student/egtea/pretrained_electra_kldiv_wt150_

### Error analysis - Instances where
* teacher - right, student-base - wrong, student-distill - right
* teacher - right, student-base - wrong, student-distill - wrong
* teacher - wrong, student-base - right, student-distill - right
* and so on

In [ ]:
res_teacher = compute_epic_metrics(path = './../teacher_pred/epic_55_3/bert_E_9_B_16_T_False_W_False_chpk_1MRecipe_MT_True_hL_5.pkl')
res_baseline = compute_epic_metrics(path = './../logit_analysis/EK55/baseline1_preds.pickle', teacher=False)
res_distil = compute_epic_metrics(path = './../logit_analysis/distillation/pretrained_alberta_kldiv_top50_wt20_t5_preds.pickle', teacher=False)

perf_diff = np.array([ res_baseline[1][k] - res_distil[1][k] for k, v in res_baseline[1].items()])
no_instances = np.array([ label_freq[k] for k, v in res_baseline[1].items()])

In [ ]:
# ----------------------------------------
# EGTEA-GAZE+ Predictions
# ----------------------------------------
# with open('./../data/egtea_action_seq/val_action_segments_egtea.pickle', "rb") as input_file:
#     egtea_input = pickle.load(input_file)

# with open('./../logit_analysis/EGTEA/baseline_preds.pickle', "rb") as input_file:
#     egtea_avt = pickle.load(input_file)

# with open('./../logit_analysis/EGTEA/kldiv_bert_wt150_preds.pickle', "rb") as input_file:
#     egtea_dist = pickle.load(input_file)

# with open('./../teacher_pred/egtea_2/bert_E_3_T_False_W_True_chpk_1MRecipe_MT_True_hL_15_SS_True_GS_True.pkl', "rb") as input_file:
#     egtea_teacher = pickle.load(input_file)

# ----------------------------------------
# EPIC KITCHEN Predictions
# ----------------------------------------
with open('./../data/epic55_action_seq/action_segments_val_ek55_complete_hist_with_uid.pickle', "rb") as input_file:
    epic_input = pickle.load(input_file)

with open('./../logit_analysis/EK55/baseline1_preds.pickle', "rb") as input_file:
    epic_avt = pickle.load(input_file)

with open('./../teacher_pred/epic_55_3/alberta_E_7_B_16_T_False_W_False_chpk_1MRecipe_MT_True_hL_5.pkl', "rb") as input_file:
    epic_teacher = pickle.load(input_file)

with open('./../logit_analysis/distillation/pretrained_alberta_kldiv_top50_wt20_t5_preds.pickle', "rb") as input_file:
    epic_avt_plus_teacher = pickle.load(input_file)

epic_avt = {uid: {'logit_avt': logit, 'target': target} for uid, logit, target in zip(epic_avt['uid'], epic_avt['logits'], epic_avt['target'])}
epic_teacher = {dict['uid']: dict['logit_action'] for dict in epic_teacher}
epic_avt_plus_teacher = {uid: {'logit_dist': logit} for uid, logit in zip(epic_avt_plus_teacher['uid'], epic_avt_plus_teacher['logits'])}

In [ ]:
epic_pred = [{'query_uid': dict['uid'],
            'label_history': [labels_mapping[x]['label'] for x in dict['history'][0]], 
            'uid_history': dict['uid_history'][0], 
            'target': dict['target'], 
            'logit_avt': epic_avt[dict['uid']]['logit_avt'],
            'pred_avt': np.argmax(epic_avt[dict['uid']]['logit_avt']),
            'pred_avt_top5': np.argsort(-epic_avt[dict['uid']]['logit_avt'])[:5],
            'logit_avt_plus_teacher': epic_avt_plus_teacher[dict['uid']]['logit_dist'],
            'pred_avt_plus_teacher': np.argmax(epic_avt_plus_teacher[dict['uid']]['logit_dist']),
            'pred_avt_plus_teacher_top5': np.argsort(-epic_avt_plus_teacher[dict['uid']]['logit_dist'])[:5],
            'logit_teacher': epic_teacher[dict['uid']].reshape(-1),
            'pred_teacher': np.argmax(epic_teacher[dict['uid']].reshape(-1)),
            'pred_teacher_top5': np.argsort(-epic_teacher[dict['uid']].reshape(-1))[:5]} 
            for dict in epic_input
            if dict['uid'] in epic_avt.keys()]
epic_pred_df = pd.DataFrame.from_dict(epic_pred, orient='columns')

In [ ]:
mappings = pd.read_csv('./../epic/epic-kitchens-55-annotations-master/EPIC_train_action_labels.csv')
epic_pred_df = epic_pred_df.merge(mappings, left_on='query_uid', right_on='uid', how='left')

In [ ]:
epic_pred_df.loc[:, 'target_txt'] = epic_pred_df.loc[:, 'target'].apply(lambda x: labels_mapping[x]['label'])
epic_pred_df.loc[:, 'pred_avt_top5_txt'] = epic_pred_df.loc[:, 'pred_avt_top5'].apply(lambda x: [labels_mapping[y]['label'] for y in x])
epic_pred_df.loc[:, 'pred_teacher_top5_txt'] = epic_pred_df.loc[:, 'pred_teacher_top5'].apply(lambda x: [labels_mapping[y]['label'] for y in x])
epic_pred_df.loc[:, 'pred_avt_plus_teacher_top5_txt'] = epic_pred_df.loc[:, 'pred_avt_plus_teacher_top5'].apply(lambda x: [labels_mapping[y]['label'] for y in x])

In [ ]:
epic_pred_df_base_w_teach_c_distil_c = epic_pred_df.loc[epic_pred_df.apply(lambda x: (x['target'] not in np.array(x['pred_avt_top5']))
                                                                                     and (x['target'] in np.array(x['pred_teacher_top5']))
                                                                                     and (x['target'] in np.array(x['pred_avt_plus_teacher_top5']))
                                                                                    , axis = 1), :]
epic_pred_df_base_w_teach_c_distil_c.loc[:, 'error_type'] = 'base_w_teach_c_distil_c'

epic_pred_df_base_w_teach_c_distil_w = epic_pred_df.loc[epic_pred_df.apply(lambda x: (x['target'] not in np.array(x['pred_avt_top5']))
                                                                                     and (x['target'] in np.array(x['pred_teacher_top5']))
                                                                                     and (x['target'] not in np.array(x['pred_avt_plus_teacher_top5']))
                                                                                    , axis = 1), :]
epic_pred_df_base_w_teach_c_distil_w.loc[:, 'error_type'] = 'base_w_teach_c_distil_w'

epic_pred_df_base_w_teach_w_distil_w = epic_pred_df.loc[epic_pred_df.apply(lambda x: (x['target'] not in np.array(x['pred_avt_top5']))
                                                                                     and (x['target'] not in np.array(x['pred_teacher_top5']))
                                                                                     and (x['target'] not in np.array(x['pred_avt_plus_teacher_top5']))
                                                                                    , axis = 1), :]
epic_pred_df_base_w_teach_w_distil_w.loc[:, 'error_type'] = 'base_w_teach_w_distil_w'

epic_pred_df_base_c_teach_w_distil_w = epic_pred_df.loc[epic_pred_df.apply(lambda x: (x['target'] in np.array(x['pred_avt_top5']))
                                                                                     and (x['target'] not in np.array(x['pred_teacher_top5']))
                                                                                     and (x['target'] not in np.array(x['pred_avt_plus_teacher_top5']))
                                                                                    , axis = 1), :]
epic_pred_df_base_c_teach_w_distil_w.loc[:, 'error_type'] = 'base_c_teach_w_distil_w'

In [ ]:
error_df = pd.concat([epic_pred_df_base_w_teach_c_distil_c, epic_pred_df_base_w_teach_c_distil_w, epic_pred_df_base_w_teach_w_distil_w, epic_pred_df_base_c_teach_w_distil_w], axis=0)
error_df.to_csv('./../examples/error.csv', index=False)
epic_pred_df.to_csv('./../examples/full_df.csv', index = False)

In [ ]:
epic_pred_df_base_w_teach_c_distil_c.to_csv('./../examples/epic_pred_df_base_w_teach_c_distil_c.csv', index=False)
epic_pred_df_base_w_teach_c_distil_w.to_csv('./../examples/epic_pred_df_base_w_teach_c_distil_w.csv', index=False)
epic_pred_df_base_w_teach_w_distil_w.to_csv('./../examples/epic_pred_df_base_w_teach_w_distil_w.csv', index=False)
epic_pred_df_base_c_teach_w_distil_w.to_csv('./../examples/epic_pred_df_base_c_teach_w_distil_w.csv', index=False)
epic_pred_df.to_csv('./../examples/epic_pred_df.csv', index=False)
epic_pred_df_base_w_teach_c_distil_c.shape, epic_pred_df_base_w_teach_c_distil_w.shape, epic_pred_df_base_w_teach_w_distil_w.shape, epic_pred_df_base_c_teach_w_distil_w.shape

### <u>Performance computation for both teacher & student</u>

In [ ]:
def compute_epic_metrics(path, teacher = True):
    with open(path, "rb") as input_file:
        epic_pred = pickle.load(input_file)

    if teacher == False:
        logit = epic_pred['logits']
        pred = logit.argmax( axis  = 1)
        target = epic_pred['target']
    else:
        target = np.array([dict['target'] for dict in epic_pred])
        logit = np.concatenate([dict['logit_action'].reshape(1, -1) for dict in epic_pred])
        pred = np.argmax(logit, axis = 1)

    K = 5
    class_recall_list= []
    class_recall_dict = {}

    acc = np.mean(pred == target)
    top_K_acc = np.mean([t in (-p).argsort()[:K] for t, p in zip(target, logit)])

    many_shot_labels_in_test = np.array(list(set(many_shot_labels).intersection(set(target))))
    for c in many_shot_labels_in_test:
        target_subset = target[target == c]
        logit_subset = logit[target == c]
        no_of_class_instances = len(target_subset)

        recall = np.mean([t in (-l).argsort()[:K]  for t, l in zip(target_subset, logit_subset)])
        class_recall_dict[c] = recall

        class_recall_list.append(recall)

    many_shot_recall = np.mean(class_recall_list)
    res_dict = {'Top_1_acc': acc, 'top_5_acc' :top_K_acc, 'm_shot_cm_recall' : many_shot_recall}
    print(path)

    conf_matrix = confusion_matrix(target, pred)
    class_wise_acc = conf_matrix.diagonal()/conf_matrix.sum(1)
    mean_class_acc = np.nanmean(class_wise_acc)

    print(f'Top 1 accuracy is :{mean_class_acc}')
    print(f'Top 5 accuracy is :{acc}, top-5 accuracy is :{top_K_acc}, many-shot top@5 class-recall : {many_shot_recall}\n')
    return res_dict, class_recall_dict

In [ ]:
from collections import Counter

with open('./../teacher_pred/epic_55_3/bert_E_9_B_16_T_True_W_False_chpk_1MRecipe_MT_True_hL_5.pkl', "rb") as input_file:
    tr_dat = pickle.load(input_file)
tr_labels = [v['target'] for v in tr_dat]
label_freq = Counter(tr_labels)

In [ ]:
res_teacher = compute_epic_metrics(path = './../teacher_pred/epic_55_3/bert_E_9_B_16_T_False_W_False_chpk_1MRecipe_MT_True_hL_5.pkl')
res_baseline = compute_epic_metrics(path = './../logit_analysis/EK55/baseline1_preds.pickle', teacher=False)
res_distil = compute_epic_metrics(path = './../logit_analysis/distillation/pretrained_alberta_kldiv_top50_wt20_t5_preds.pickle', teacher=False)

perf_diff = np.array([ res_baseline[1][k] - res_distil[1][k] for k, v in res_baseline[1].items()])
no_instances = np.array([ label_freq[k] for k, v in res_baseline[1].items()])

In [ ]:
teacher_pred_path = "epic"   # or epic/egtea
model_name = ['alberta', 'bert', 'roberta', 'electra', 'distillbert']
for model in model_name:
    file_name = f'./../teacher_student_Predictions/teacher/{teacher_pred_path}/{model}_E_7_B_16_T_False_W_False_chpk_1MRecipe_MT_True_hL_5.pkl'
    # file_name = f'./../teacher_student_Predictions/teacher/{teacher_pred_path}/{model}_E_5_T_False_W_True_chpk_1MRecipe_MT_True_hL_15_SS_True_GS_True.pkl'
    compute_epic_metrics(path=file_name, teacher=True)


### Compute performance

### Covariance analysis

In [ ]:
logit_corr = np.corrcoef(torch.softmax(torch.tensor(logit), dim=0).numpy().transpose())

In [ ]:
corr_analysis_dict = []
for k, v in labels_mapping_manyshot.items():
    dict_temp = {}
    dict_temp['label'] = v['label']

    corr_row = (-logit_corr[k, :]).argsort()[1:6]
    corr_row = [labels_mapping[label_id]['label'] for label_id in corr_row]
    dict_temp['correlated_labels'] = corr_row
    corr_analysis_dict.append(dict_temp)
pd.DataFrame(corr_analysis_dict).to_csv('./../logit_analysis/base_student_correlated_labels.csv', index = False)

In [ ]:
print(pd.DataFrame(corr_analysis_dict))